# Building and runing a web crawler.

In [6]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

The basic url. other pages can be visited by incrementing `page=;`

In [5]:
# base_url contains a placeholder {} for the page number. 
base_url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=1;orderby=runs;page={};template=results;type=batting'

**Number of pages =?**

In [21]:
n_pages=5

**Looping through all the pages**

In [31]:
for pagenum in range(1, n_pages+1):
    
    print(f'Visiting page number {pagenum}. ', end='')
    # updating url for page number
    url = base_url.format(pagenum)
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Response status code = {response.status_code} !!')
        break
    
    soup = BeautifulSoup(response.content, 'html5lib')
    table_class = 'engineTable'
    tables = soup.find_all('table', class_=table_class)
    desired_table=None
    for table in tables:
        if not (table.has_attr('style')):
            desired_table = table
            break
    table = desired_table

    t_head = table.find('thead')
    t_body = table.find('tbody')

    if pagenum==1:
        cols = dict()
        for tr in t_head.find_all('tr'):
            for th in t_head.find_all('th'):
                cols[th.text.strip()] = None

        cols.pop('') # this column is given to include links
        # DataFrame part
        df = pd.DataFrame(cols, index=[0])
    
    for row in t_body.find_all('tr'):
        for col, td in zip(cols.keys() ,row.find_all('td')):
            if td.text.strip() == 'No records available to match this query': break
            cols[col] = td.text.strip()
        df_new_entry = pd.DataFrame(cols, index=[0])
        df = pd.concat([df, df_new_entry], ignore_index=True)
    
    print(f'Page {pagenum} parsed.')

Visiting page number 1. Page 1 parsed.
Visiting page number 2. Page 2 parsed.
Visiting page number 3. Page 3 parsed.
Visiting page number 4. Page 4 parsed.
Visiting page number 5. Page 5 parsed.


In [32]:
df[-25:]

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,100,50,0
226,GR Marsh (AUS),1985-1992,50,93,7,2854,138,33.18,4,15,3
227,Ramiz Raja (PAK),1984-1997,57,94,5,2833,122,31.83,2,22,7
228,Aamer Sohail (PAK),1992-2000,47,83,3,2823,205,35.28,5,13,6
229,Wasim Raja (PAK),1973-1985,57,92,14,2821,125,36.16,4,18,8
230,KR Stackpole (AUS),1966-1974,43,80,5,2807,207,37.42,7,14,5
231,RA Jadeja (IND),2012-2023,67,98,21,2804,175*,36.41,3,19,6
232,JO Holder (WI),2014-2023,64,113,18,2797,202*,29.44,3,12,9
233,MAK Pataudi (IND),1961-1975,46,83,3,2793,203*,34.91,6,16,7
234,KC Wessels (AUS/SA),1982-1994,40,71,3,2788,179,41.00,6,15,5
235,NCL O'Neill (AUS),1958-1965,42,69,8,2779,181,45.55,6,15,6


Sending the HTTP request to fetch the webpage

Success! Making the soup instance of the fetched webpage

Extracting the desired table

In [5]:
# Name of the table's class
table_class = 'engineTable'

# returns all tables with `table_class`
tables = soup.find_all('table', class_=table_class)
tables

[<table class="engineTable" style="margin:0px;">
 
  <tbody><tr class="data2">
   <td class="left">
    <b>View</b> overall figures
 [<a href="javascript:void(0)" onmouseout="menuLayers.hide()" onmouseover="menuLayers.show('engine-dd-view', event); window.status='change view'; return true">change view</a>]
 
   </td>
  </tr>
  <tr class="data2">
   <td class="left">
    <b>Ordered by</b> runs scored (descending)
 
   </td>
  </tr></tbody></table>,
 <table class="engineTable" style="margin-bottom:5px;">
  <tbody><tr class="data2">
   <td class="left" style="vertical-align:middle">
    Page <b>1</b> of <b>63</b>
   </td>
   <td class="left" style="vertical-align:middle">
    Showing <b>1</b> - <b>50</b> of <b>3142</b>
   </td>
   <td class="left" style="vertical-align:middle">
   </td><td class="left" style="vertical-align:middle">
    <img align="texttop" alt="First page" border="0" height="13" src="http://i.imgci.com/espncricinfo/ciPhotoFirst-icon_13x13.gif" style="padding-right:5px;" 

In [6]:
# Finding the table which doesnt have style attribute (thats our table)
desired_table=None
for table in tables:
    if not (table.has_attr('style')):
        desired_table = table
        break
table = desired_table
table

<table class="engineTable">
<caption>Overall figures</caption>
<thead>
 <tr class="headlinks">
  <th class="left" nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=1;orderby=player;template=results;type=batting" title="sort by player name">Player</a></th>
  <th class="left" nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=1;orderby=start;template=results;type=batting" title="sort by start date">Span</a></th>
  <th nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=1;orderby=matches;template=results;type=batting" title="sort by matches played">Mat</a></th>
  <th nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=1;orderby=innings;template=results;type=batting" title="sort by innings batted">Inns</a></th>
  <th nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=1;orderby=notouts;template=results;type=batting" title="sort by not outs">NO</a></th>
  <th nowrap=""><a class="black-link

The desired table is fetched!! Extracting the table columns headings

In [7]:
# Seperating thead and tbody
t_head = table.find('thead')
t_body = table.find('tbody')

# Making all the columns as an dictionary for now, values as None
cols = dict()
for tr in t_head.find_all('tr'):
    for th in t_head.find_all('th'):
        cols[th.text.strip()] = None

cols.pop('') # this column is given to include links
cols

{'Player': None,
 'Span': None,
 'Mat': None,
 'Inns': None,
 'NO': None,
 'Runs': None,
 'HS': None,
 'Ave': None,
 '100': None,
 '50': None,
 '0': None}

Making this dict into a **pandas dataframe** now

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(cols, index=range(50))
df

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,100,50,0
0,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,None


Now making the entries in the 1st page of the table (50)

In [10]:
for i, tr in enumerate(t_body.find_all('tr')):
    for col, td in zip(cols.keys() ,tr.find_all('td')):
        cols[col] = td.text.strip()
        df.iloc[i] = cols

In [11]:
df

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,100,50,0
0,SR Tendulkar (IND),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (ICC/IND),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
5,KC Sangakkara (SL),2000-2015,134,233,17,12400,319,57.40,38,52,11
6,BC Lara (ICC/WI),1990-2006,131,232,6,11953,400*,52.88,34,48,17
7,S Chanderpaul (WI),1994-2015,164,280,49,11867,203*,51.37,30,66,15
8,DPMD Jayawardene (SL),1997-2014,149,252,15,11814,374,49.84,34,50,15
9,JE Root (ENG),2012-2023,135,247,20,11416,254,50.29,30,60,12


Eureka !! now limiting the table to 10 entries, and manipulating the span, and team columns

In [12]:
df_10 = df[:10].copy()
df_10

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,100,50,0
0,SR Tendulkar (IND),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (ICC/IND),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
5,KC Sangakkara (SL),2000-2015,134,233,17,12400,319,57.40,38,52,11
6,BC Lara (ICC/WI),1990-2006,131,232,6,11953,400*,52.88,34,48,17
7,S Chanderpaul (WI),1994-2015,164,280,49,11867,203*,51.37,30,66,15
8,DPMD Jayawardene (SL),1997-2014,149,252,15,11814,374,49.84,34,50,15
9,JE Root (ENG),2012-2023,135,247,20,11416,254,50.29,30,60,12


Creating functtions for 
1. Seperating names and teams
2. removing span and adding start and end year

In [13]:
def span_split(df, inplace=True):
    if not inplace: df = df.copy()
    Start, End = pd.Series(index=range(len(df))), pd.Series(index=range(len(df)))
    for i in range(len(df)):
        span = df['Span'].iloc[i]
        start_year, end_year = span.split('-')
        Start.iloc[i] = start_year
        End.iloc[i] = end_year
    
    df['Start'] = Start
    df['End'] = End
    df.drop('Span', axis=1, inplace=True)
    if not inplace: return df
    return None

def player_team_split(df, inplace=True):
    
    if not inplace: df = df.copy()
    Team = pd.Series(index=range(len(df)))
    for i in range(len(df)):
        player = df['Player'].iloc[i]
        start_paren = player.find('(')
        if player.find('/')>=0: start_char = player.find('/')
        else: start_char = start_paren
        end_char = player.find(')')

        player_name = player[:start_paren-1]
        player_team = player[start_char+1 :end_char]
    
        df['Player'].iloc[i] = player_name
        Team.iloc[i] = player_team
    
    df['Team'] = Team
    if not inplace: return df
    return None

Doing the seperation task

Combining into 1 function and reordering columns

In [14]:
def df_transform(df, inplace=True):
    if not inplace:
        df = df.copy()
    df = player_team_split(df, inplace)
    df = span_split(df, inplace)
#     Reordering columns
    df = df[['Player','Team' ,'Start' ,'End' ,'Mat', 'Inns' ,'NO' ,'Runs' ,'HS' ,'Ave' ,'100' ,'50' ,'0']]
    
    if not inplace: return df

In [15]:
df_transform(df, inplace=0)

,Player,Team,Start,End,Mat,Inns,NO,Runs,HS,Ave,100,50,0
0,SR Tendulkar,IND,1989,2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting,AUS,1995,2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis,SA,1995,2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid,IND,1996,2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook,ENG,2006,2018,161,291,16,12472,294,45.35,33,57,9
5,KC Sangakkara,SL,2000,2015,134,233,17,12400,319,57.40,38,52,11
6,BC Lara,WI,1990,2006,131,232,6,11953,400*,52.88,34,48,17
7,S Chanderpaul,WI,1994,2015,164,280,49,11867,203*,51.37,30,66,15
8,DPMD Jayawardene,SL,1997,2014,149,252,15,11814,374,49.84,34,50,15
9,JE Root,ENG,2012,2023,135,247,20,11416,254,50.29,30,60,12


In [16]:
df

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,100,50,0
0,SR Tendulkar (IND),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,R Dravid (ICC/IND),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
5,KC Sangakkara (SL),2000-2015,134,233,17,12400,319,57.40,38,52,11
6,BC Lara (ICC/WI),1990-2006,131,232,6,11953,400*,52.88,34,48,17
7,S Chanderpaul (WI),1994-2015,164,280,49,11867,203*,51.37,30,66,15
8,DPMD Jayawardene (SL),1997-2014,149,252,15,11814,374,49.84,34,50,15
9,JE Root (ENG),2012-2023,135,247,20,11416,254,50.29,30,60,12
